<a href="https://colab.research.google.com/github/rahiakela/transfer-learning-for-natural-language-processing/blob/main/3-shallow-transfer-learning-for-nlp/3_multi_task_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Multi-Task Learning

In this notebook, we will cover some prominent shallow transfer learning approaches and concepts. This allows us to explore some major themes in transfer learning, while doing so in the context of relatively simple models of the class of eventual interest, i.e., shallow neural networks.

Roughly speaking, categorization is based on whether transfer occurs between different languages, tasks or data domains. Each of these types of categorization is usually correspondingly referred to as cross-lingual learning, multi-task learning and domain adaptation.

<img src='https://github.com/rahiakela/img-repo/blob/master/transfer-learning-for-natural-language-processing/shallow-transfer-learning.png?raw=1' width='800'/>

The methods we will look at here will involve components that are neural networks in one way or another.these neural networks do not have many
layers. This is the reason why the label “shallow” is appropriate to describe this collection of methods.

A common form of semi-supervised learning that employs pretrained word embeddings such as word2vec that they produce a single vector per word, regardless of context.

We revisit the IMDB movie review sentiment classification. Recall that this example is concerned with classifying movie reviews from IMDB into positive or negative sentiments expressed. It is a prototypical sentiment analysis example that has been used widely in the literature to study many algorithms. We combine feature vectors generated by pretrained word embeddings for each review with some traditional machine learning classification methods, namely random forests and logistic regression.

We then demonstrate that using higher-level embeddings which vectorize bigger sections of text – such as at the sentence-level, paragraphlevel and document-level – can lead to improved performance. The general idea of vectorizing text and then applying a traditional machine learning classification method to the resulting vectors.

<img src='https://github.com/rahiakela/img-repo/blob/master/transfer-learning-for-natural-language-processing/semi-supervised-learning.png?raw=1' width='800'/>

**Multi-task learning**

Subsequently, we introduce the reader to multi-task learning. We demonstrate how one can train a single system simultaneously to perform multiple tasks, email spam classification and IMDB movie review sentiment analysis. 

There are several potential benefits to multi-task learning. By
training a single machine learning model for multiple tasks, a shared representation is learned on a larger and more varied collection of data from the combined data pool, which can lead to performance improvements. Moreover, it has been widely observed that this shared representation has a better ability to generalize to tasks beyond those that were trained on, and
this improvement can be achieved without any increase in model size.

Specifically, we focus on shallow neural multitask learning, where a single additional dense layer, as well as a classification layer, is trained
for each specific task in the setup. Different tasks also share a layer between them, a setup typically referred to as hard-parameter sharing.

**Domain adaptation**

Assume that we are given one source domain, which can be defined as a particular distribution of data for a specific task, and a classifier that has been trained to perform well on data in that domain for that task. The goal of domain adaptation is to modify, or adapt, data in a different target domain in such a way that the pretrained knowledge from the source domain can aid
learning in the target domain. We apply a simple autoencoding approach to “project” samples in the target domain into the source domain feature space.

An autoencoder is a system that learns to reconstruct inputs with very high accuracy, typically by encoding them into an efficient latent representation and learning to decode the said representation efficiently. They have traditionally been heavily used in model reduction applications, since the latent representation is often of smaller dimension than the original space
from which the encoding happens, and the said dimension value can also be picked to strike the right balance of computational efficiency and accuracy.

In the extreme scenario, improvements can be obtained with no labelled data in the target domain being used for training. This is typically referred to as zero-shot domain adaptation, where learning happens with no labeled
data in the target domain.


## Setup

In [1]:
import tensorflow as tf

print(tf.__version__)

2.4.1


In [ ]:
# install sent2vec
!pip install git+https://github.com/epfml/sent2vec

In [10]:
import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import re
import os
import time
import email

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import sent2vec

import matplotlib.pyplot as plt
from IPython.display import HTML

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Download IMDB Movie Review Dataset

In [40]:
%%shell

wget -q "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
tar xzf aclImdb_v1.tar.gz

rm -rf aclImdb_v1.tar.gz
rm -rf aclImdb/train/unsup

Let's download sent2vec word Embedding from [Kaggle](https://www.kaggle.com/maxjeblick/sent2vec)

In [4]:
from google.colab import files
files.upload() # upload kaggle.json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rahiakela","key":"484f91b2ebc194b0bff8ab8777c1ebff"}'}

In [5]:
%%shell

mkdir -p ~/.kaggle
mv kaggle.json ~/.kaggle/
ls ~/.kaggle
chmod 600 /root/.kaggle/kaggle.json

# download word embeddings from kaggle
kaggle datasets download -d maxjeblick/sent2vec/wiki_unigrams.bin
unzip -qq sent2vec.zip
rm -rf sent2vec.zip

# download dataset from kaggle
kaggle datasets download -d wcukierski/enron-email-dataset
unzip -qq enron-email-dataset.zip

kaggle datasets download -d rtatman/fraudulent-email-corpus
unzip -qq fraudulent-email-corpus.zip

rm -rf enron-email-dataset.zip fraudulent-email-corpus.zip

kaggle.json
100% 4.42G/4.43G [01:50<00:00, 68.1MB/s]
100% 4.43G/4.43G [01:50<00:00, 43.1MB/s]
 99% 353M/358M [00:11<00:00, 39.6MB/s]
100% 358M/358M [00:11<00:00, 32.9MB/s]
 91% 5.00M/5.52M [00:00<00:00, 12.8MB/s]
100% 5.52M/5.52M [00:00<00:00, 13.9MB/s]


In [6]:
def extract_messages(df):
  messages = []
  for item in df["message"]:
    # Return a message object structure from a string
    e = email.message_from_string(item)
    # get message body
    message_body = e.get_payload()
    messages.append(message_body)
  print("Successfully retrieved message body from e-mails!")
  return messages

## Preprocessing IMDB Data

Before proceeding, we must decide how many samples to draw from each class. We must also decide the maximum number of tokens per email, and the maximum length of each token. This is done by setting the following overarching hyperparameters.

In [21]:
n_sample = 1000    # number of samples to generate in each class
maxtokens = 200    # the maximum number of tokens per document
maxtokenlen = 100  # the maximum length of each token

### Tokenization

Let’s proceed by defining a function to tokenize text by splitting them into 
words.

In [17]:
def tokenize(row):
  if row is None or row is "":
    tokens = ""
  else:
    tokens = str(row).split(" ")[:maxtokens]
  return tokens

### Remove punctuation and unnecessary characters

**In order to ensure that classification is done based on language content only, we have to remove punctuation marks and other non-word characters from the emails.** We do this by employing regular expressions with the Python regex library. We also normalize words by turning them into lower case.

In [18]:
def reg_expressions(row):
  tokens = []
  try:
    for token in row:
      token = token.lower()          # make all characters lower case
      token = re.sub(r"[\W\d]", "", token)
      token = token[:maxtokenlen]    # truncate all tokens to hyperparameter maxtokenlen
      tokens.append(token)
  except:
    token = ""
    tokens.append(token)
  return tokens

### Stop-word removal

Stop-words are also removed. Stop-words are words that are very common in text but offer no useful information that can be used to classify the text. Words such as is, and, the, are are examples of stop-words. The NLTK library contains a list of 127 English stop-words and can be used to filter our tokenized strings.

In [23]:
stop_words = stopwords.words("english")
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
# print(stopwords) # see default stopwords
# it may be beneficial to drop negation words from the removal list, as they can change the positive/negative meaning
# of a sentence
# stopwords.remove("no")
# stopwords.remove("nor")
# stopwords.remove("not")

In [19]:
def stop_word_removal(row):
  token = [token for token in row if token not in stop_words]
  token = filter(None, token)

  return token

### Load pre-trained sent2vec embedding

Quite naturally, just as in the case of the pretrained word embeddings, the next step is to obtain the pretrained sent2vec sentence embedding to be loaded by the particular implementation/framework installed.

We choose the smallest 600-dimensional embedding `wiki_unigrams.bin`, approximately 5 Gigabytes in size, which captures just the unigram information on Wikipedia.

Now let's load the pre-trained embedding.

In [46]:
# load sent2vec embedding
model = sent2vec.Sent2vecModel()

start = time.time()
model.load_model("wiki_unigrams.bin")
end = time.time()

print("Loading the sent2vec embedding took %d seconds" % (end - start))

Loading the sent2vec embedding took 142 seconds


### Extract corresponding vectors from the pretrained word embedding

Next, we define a function to generate vectors for a collection of reviews. It is essentially a simpler form of the function presented in Listing 3.2 for pretrained word embeddings – it is simpler as we do not need to worry about out-of-vocabulary words.

In [28]:
def assemble_embedding_vectors(data):
  out = None
  for item in data:    # Loop through every IMDB review
    vec = model.embed_sentence(" ".join(item))     # Extract embedding vectors for every word in review, now we dont need to handle out-of-vocab words
    if vec is not None:                            # Edge case handling
      if out is not None:
        out = np.concatenate((out, vec), axis=0)    # Concatenate row vector to output Numpy array
      else:
        out = vec
    else:
      pass

  return out

### Preparing and assembling Dataset

In [38]:
# shuffle raw data first
def unison_shuffle_data(data, header):
    p = np.random.permutation(len(header))
    data = data[p]
    header = np.asarray(header)[p]
    return data, header

# load data in appropriate form
def load_data(path):
  data, sentiments = [], []
  for folder, sentiment in (("neg", 0), ("pos", 1)):
    folder = os.path.join(path, folder)
    for name in os.listdir(folder):    # Go through every file in current folder
      with open(os.path.join(folder, name), "r") as reader:
        text = reader.read()
      # Apply tokenization, stopword analysis routines
      text = tokenize(text)
      text = stop_word_removal(text)
      text = reg_expressions(text)
      # Track corresponding text and sentiment labels
      data.append(text)
      sentiments.append(sentiment)
  # Convert to Numpy array
  #print(data)
  data_np = np.array(data)
  #print(data_np[:10])
  data, sentiments = unison_shuffle_data(data_np, sentiments)

  return data, sentiments

In [41]:
train_path = os.path.join("aclImdb", "train")
test_path = os.path.join("aclImdb", "test")
raw_data, raw_header = load_data(train_path)

print(raw_data.shape)
print(len(raw_header))

(25000,)
25000


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [42]:
# Subsample required number of samples
random_indices = np.random.choice(range(len(raw_header)), size=(n_sample * 2,), replace=False)
data_train = raw_data[random_indices]
header = raw_header[random_indices]

print("DEBUG::data_train::")
print(data_train[:10])

DEBUG::data_train::
[list(['ah', 'kellysinatra', 'sailorsuit', 'musical', 'so', 'familiar', 'right', 'yes', 'one', 'usually', 'hear', 'about', 'on', 'the', 'towns', 'thataway', 'but', 'stick', 'around', 'might', 'learn', 'something', 'okay', 'probably', 'not', 'anyway', 'anchors', 'aweigh', 'tells', 'story', 'two', 'sailors', 'three', 'fourday', 'leave', 'joe', 'sea', 'wolf', 'clarence', 'bookish', 'type', 'begs', 'joe', 'get', 'dame', 'now', 'theyre', 'picked', 'coppers', 'get', 'little', 'donald', 'home', 'thats', 'meet', 'susie', 'temptress', 'jezebel', 'just', 'kidding', 'clarence', 'falls', 'love', 'her', 'at', 'least', 'thinks', 'does', 'is', 'right', 'or', 'moron', 'or', 'misguided', 'society', 'find', 'watch', 'trumpet', 'fanfare', 'anchors', 'aweighbr', 'br', 'ps', 'if', 'want', 'see', 'kathryn', 'grayson', 'anything', 'sickeningly', 'sweet', 'try', 'kiss', 'me', 'kate', ''])
 list(['while', 'people', 'watching', 'movie', 'little', 'interest', 'many', 'hundreds', 'movies', 'de

Display sentiments and their frequencies in the dataset, to ensure it is roughly balanced between classes.

In [43]:
unique_elements, counts_elements = np.unique(header, return_counts=True)

print("Sentiments and their frequencies:")
print(unique_elements)
print(counts_elements)

Sentiments and their frequencies:
[0 1]
[ 958 1042]


We can now use this function to extract sent2vec embedding vectors for each review.

In [47]:
embedding_vectors = assemble_embedding_vectors(data_train) 
print(embedding_vectors)

[[ 0.03518312 -0.20093317  0.05013918 ...  0.08352327 -0.06436302
   0.09644231]
 [-0.13478053 -0.24570955  0.01786834 ... -0.06769671  0.00131936
   0.1521685 ]
 [ 0.07780235  0.0006177   0.00827089 ...  0.01655485  0.08689536
   0.00155434]
 ...
 [ 0.02962787 -0.04588524 -0.07604289 ...  0.07706061 -0.13126795
   0.18768099]
 [ 0.12826902 -0.07406035  0.03565962 ... -0.09084704 -0.14573774
   0.12216499]
 [-0.17812063  0.08628934 -0.23646048 ...  0.1671205  -0.22869939
  -0.14114092]]


These can now be used as feature vectors for the same logistic regression and random forest.

As the very last step of preparing the sentiment dataset for training by our baseline classifiers, we split it into independent training and testing or validation sets. This will allow us to evaluate the performance of the classifier on a set of data that was not used for training, an important thing
to ensure in machine learning practice. We elect to use 70% of the data for training, and 30% for testing/validation afterwards.

In [48]:
data = embedding_vectors
del embedding_vectors

idx = int(0.7 * data.shape[0])


# 70% of data for training
train_x = data[:idx, :]
train_y = header[:idx]

# remaining 30% for testing
test_x = data[idx:, :]
test_y = header[idx:]

print("train_x/train_y list details, to make sure it is of the right form:")
print(len(train_x))
print(train_x[:5])
print(len(train_y))
print(train_y[:5])

train_x/train_y list details, to make sure it is of the right form:
1400
[[ 0.03518312 -0.20093317  0.05013918 ...  0.08352327 -0.06436302
   0.09644231]
 [-0.13478053 -0.24570955  0.01786834 ... -0.06769671  0.00131936
   0.1521685 ]
 [ 0.07780235  0.0006177   0.00827089 ...  0.01655485  0.08689536
   0.00155434]
 [-0.05747809 -0.06960996  0.1839919  ...  0.17964846 -0.04227288
   0.01319464]
 [ 0.03982377 -0.20946196 -0.04318017 ... -0.02148014  0.07441714
   0.02773258]]
1400
[1 1 1 1 1]


## Preprocessing Email Spam Data

### Loading and Visualizing the Fraudulent Email Corpus

In [7]:
filepath = "./fradulent_emails.txt"
with open(filepath, "r", encoding="latin1") as file:
  data = file.read()

In [8]:
fraud_emails = data.split("From r")
del data

print("Successfully loaded {} spam emails!".format(len(fraud_emails)))

Successfully loaded 3978 spam emails!


In [11]:
fraud_bodies = extract_messages(pd.DataFrame(fraud_emails, columns=["message"]))
del fraud_emails

fraud_bodies_df = pd.DataFrame(fraud_bodies[1:])
del fraud_bodies

fraud_bodies_df.head()

Successfully retrieved message body from e-mails!


,0
0,FROM:MR. JAMES NGOLA.\nCONFIDENTIAL TEL: 233-2...
1,"Dear Friend,\n\nI am Mr. Ben Suleman a custom ..."
2,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...
3,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...
4,"Dear sir, \n \nIt is with a heart full of hope..."


### Loading and Visualizing the Enron Corpus

In [12]:
filepath = "./emails.csv"

# Read the enron data into a pandas.DataFrame called emails
emails = pd.read_csv(filepath)
print("Successfully loaded {} rows and {} columns!".format(emails.shape[0], emails.shape[1]))
print(emails.head())

Successfully loaded 517401 rows and 2 columns!
                       file                                            message
0     allen-p/_sent_mail/1.  Message-ID: <18782981.1075855378110.JavaMail.e...
1    allen-p/_sent_mail/10.  Message-ID: <15464986.1075855378456.JavaMail.e...
2   allen-p/_sent_mail/100.  Message-ID: <24216240.1075855687451.JavaMail.e...
3  allen-p/_sent_mail/1000.  Message-ID: <13505866.1075863688222.JavaMail.e...
4  allen-p/_sent_mail/1001.  Message-ID: <30922949.1075863688243.JavaMail.e...


In [13]:
# take a closer look at the first email
print(emails.loc[0]["message"])

Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 


In [14]:
bodies = extract_messages(emails)

# no longer needed, get rid of them
del emails

Successfully retrieved message body from e-mails!


In [15]:
# extract random 10000 enron email bodies for building dataset
bodies_df = pd.DataFrame(random.sample(bodies, 10000))
# these are huge, no longer needed, get rid of them
del bodies 

# expand default pandas display options to make emails more clearly visible when printed
pd.set_option("display.max_colwidth", 300)
# you could do print(bodies_df.head()), but Jupyter displays this nicer for pandas DataFrames
bodies_df.head()

,0
0,"------------------------------------------------------------------------------\n------------------------\nW E E K E N D S Y S T E M S A V A I L A B I L I T Y\n\nF O R\n\nMay 11, 2001 5:00pm through May 14, 2001 12:00am\n------------------------------------------------------------------------..."
1,"Louise would like for you to be in attendance at the PHASE 2 meeting \nscheduled for Thursday & Friday, February 10th & 11th. These meetings will \nconvene at the Doubletree Hotel beginning at 8:00 a.m. on Thursday morning. \nIf you are unable to attend for any reason, please contact me righ..."
2,"Chris:\nThanks for your prompt response. I think that we do need to register the \ncharge document which in this case would be the confirmation that will be \nexecuted. Can you provide me with an estimate of the total cost?\n\nDoes everyone at Enron agree?\n\nCaroline,\nI will redraft the Bermu..."
3,"\nHi Jonathan, \nA couple of questions for you: \n1) Have you proceeded with opening an account at Dane Rauscher yet? \n2) Your RSP account has a small debit of $50.00, probably due to RSP holding fees charged to the account for 2001. We either need a cheque from you for the $50.00 or we need t..."
4,"Attached please find the latest versions of the above-referenced documents, \nmarked to show changes from the versions distributed 12/9, except in the case \nof Annexes C ( Guaranty and Indemnification Agreement) and D ( form of the \ndrawing certificate to the CCSI letter of credit, the complet..."


### Preparing and assembling Dataset

We are now going to put all these functions together to build the single dataset representing both classes. Most methods expect this dataset to be a Numpy array in order to process it, so we convert it to that form after combining the emails.

Now, putting all the preprocessing steps together we assemble our dataset...

In [24]:
# Convert everything to lower-case, truncate to maxtokens and truncate each token to maxtokenlen

# Apply predefined processing functions
enron_emails = bodies_df.iloc[:, 0].apply(tokenize)
enron_emails = enron_emails.apply(stop_word_removal)
enron_emails = enron_emails.apply(reg_expressions)
# sample the right number of emails from each class.
enron_emails = enron_emails.sample(n_sample)

del bodies_df

# Apply predefined processing functions
spam_emails = fraud_bodies_df.iloc[:, 0].apply(tokenize)
spam_emails = spam_emails.apply(stop_word_removal)
spam_emails = spam_emails.apply(reg_expressions)
# sample the right number of emails from each class.
spam_emails = spam_emails.sample(n_sample)

del fraud_bodies_df

# convert to Numpy array
raw_data = pd.concat([enron_emails, spam_emails], axis=0).values

Now, let’s take a peek at the result to make sure things are proceeding as expected:

In [25]:
print("Shape of combined data is:", raw_data.shape)
print("Data is:")
print(raw_data)

Shape of combined data is: (2000,)
Data is:
[list(['louise', 'may', 'aware', 'vng', 'agreement', 'provides', 'annual', 'payment', 'shared', 'value', 'sequent', 'energy', 'november', '', 'this', 'years', 'payment', 'scheduled', 'made', 'november', '', '', 'amount', '', 'given', 'previous', 'discussions', 'i', 'felt', 'appropriate', 'give', 'heads', 'matterfrank'])
 list(['have', 'trying', 'run', 'gordon', 'white', 'several', 'days', 'regarding', 'ameeting', 'finally', 'talked', 'today', 'he', 'town', 'north', 'carolinathe', 'rest', 'week', 'leaves', 'sweden', 'almost', 'two', 'weeks', 'thisweekend', 'the', 'earliest', 'date', 'could', 'set', 'meeting', 'accomodates', 'hisschedule', 'wednesday', 'dec', 'th', 'gordons', 'offices', 'austin', 'i', 'wanted', 'doit', 'earlier', 'gordon', 'essentially', 'pocket', 'almost', 'three', 'weeksdoes', 'date', 'work', 'you', 'confidentiality', 'noticethe', 'information', 'email', 'may', 'confidential', 'andor', 'privileged', 'thisemail', 'intended', '

We see that the resulting array has divided the text into word units, as we intended to.

Let’s create the headers corresponding to these emails, consisting of n_sample=1000 of spam emails followed by n_sample=1000 of non-spam emails:

In [26]:
categories = ["spam", "notspam"]
header = ([1] * n_sample)
header.extend(([0] * n_sample)) 

We are now ready to convert these into numerical vectors!!

In [31]:
embedding_vectors = assemble_embedding_vectors(raw_data)
print(embedding_vectors)

[[-0.13003325 -0.05259239  0.05535318 ... -0.06819534 -0.30347332
   0.17158867]
 [-0.20624852 -0.15659373  0.17093782 ...  0.00211835 -0.09384697
  -0.15397482]
 [-0.26698807 -0.22275618  0.21200468 ...  0.0697145  -0.03027352
  -0.0491537 ]
 ...
 [ 0.04502249 -0.10238895  0.0200396  ...  0.1996169  -0.00941314
   0.06404965]
 [-0.01586943 -0.02585013 -0.00210908 ...  0.00710615 -0.12967923
  -0.12611614]
 [-1.1175805  -0.751737    0.55355763 ... -0.6607673   0.21230514
  -0.30602062]]


In [32]:
# shuffle raw data first
def unison_shuffle_data(data, header):
  p = np.random.permutation(len(header))
  data = data[p,:]
  header = np.asarray(header)[p]
  return data, header

In [33]:
raw_data, header = unison_shuffle_data(embedding_vectors, header)

# split into independent 70% training and 30% testing sets
idx = int(0.7 * raw_data.shape[0])  # get 70% index value

# 70% of data for training
train_x2 = raw_data[:idx, :]
train_y2 = header[:idx]

# remaining 30% for testing
test_x2 = raw_data[idx:, :]
test_y2 = header[idx:]

print("train_x2/train_y2 list details, to make sure they are of the right form:")
print(len(train_x2))
print(train_x2)
print(len(train_y2))
print(train_y2[:5])

train_x2/train_y2 list details, to make sure they are of the right form:
1400
[[-0.0844844  -0.02524753 -0.04236837 ...  0.04642141 -0.17143312
  -0.06707019]
 [-0.04295615  0.01730135  0.04939496 ... -0.0560524  -0.00499381
  -0.04114064]
 [ 0.00597835 -0.11049199 -0.02723659 ...  0.13010345 -0.244281
  -0.01854577]
 ...
 [-0.16114262  0.11494704  0.52283984 ... -0.1570149   0.00378184
   0.09888405]
 [ 0.02190978 -0.09102036  0.31848645 ...  0.08235449 -0.09449536
   0.13476945]
 [-1.1175805  -0.751737    0.55355763 ... -0.6607673   0.21230514
  -0.30602062]]
1400
[0 0 0 0 0]


## Multi-Task Learning

Traditionally, **machine learning algorithms have been trained to perform a single task at a time, with the data collected and trained on independent for each separate task.** This is somewhat antithetical to the way humans and other animals learn, where training for multiple tasks occurs
simultaneously and information from training on one task may inform and accelerate learning of other different tasks. **This additional information may improve performance not just on the current tasks being trained on, but also on future tasks, and sometimes even in cases where no labeled data is available on such future tasks. This scenario of transfer learning with no labeled data in the target domain is often referred to as zero-shot transfer learning.**

In machine learning, multi-task learning has historically appeared in a number of settings – from multi-objective optimization to l2 and other forms of regularization (which itself can be framed as a form of multi-objective optimization).

<img src='https://github.com/rahiakela/img-repo/blob/master/transfer-learning-for-natural-language-processing/neural-multi-task-learning.png?raw=1' width='800'/>

In the other prominent type of neural multi-task learning, soft parameter sharing, all tasks have their own layers/parameters, which are not shared. Instead, they are encouraged to be similar via various constraints imposed on the task-specific layers across the various tasks.



## Problem Setup and Shallow Neural Single-Task Baseline

Let's consider now again, with 2 tasks only, the first task being IMDB movie review classification and the second task being email spam classification.

<img src='https://github.com/rahiakela/img-repo/blob/master/transfer-learning-for-natural-language-processing/neural-multi-task-hard-parameter-sharing.png?raw=1' width='800'/>

Before proceeding, we must decide how the inputs to the resulting neural network will be converted into numbers for analysis. One popular choice is to encode the input at the characterlevel using one-hot encoding, where each character is replaced by a sparse vector of dimension equal to the total number of possible characters. This vector contains 1 in the column
corresponding to the character and 0 otherwise. An illustration of this method, which aims to help the reader concisely visualize the process of one-hot encoding.

<img src='https://github.com/rahiakela/img-repo/blob/master/transfer-learning-for-natural-language-processing/one-hot-encoding-characters.png?raw=1' width='800'/>

Before proceeding to the exact two-task setup, we perform another
baseline. We use the IMDB movie classification task as the only one present, to see how the task-specific shallow neural classifier compares with the model from the previous section.


In [35]:
input_shape = (len(train_x2[0]), )

# Input must match the dimension of the sent2vec vectors
sent2vec_vectors = Input(shape=input_shape)
# Dense neural layer trained on top of the sent2vec vectors
dense = Dense(512, activation="relu")(sent2vec_vectors)
# Apply dropout to reduce overfitting
dense = Dropout(0.3)(dense)

# Output indicates a single binary classifier - is review “positive” or “negative”?
output = Dense(1, activation="sigmoid")(dense)

model = Model(inputs=sent2vec_vectors, outputs=output)

In [36]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_x2, train_y2, validation_data=(test_x2, test_y2), batch_size=32, epochs=10, shuffle=True)

Epoch 1/10
44/44 [==============================] - 3s 10ms/step - loss: 0.3392 - accuracy: 0.8926 - val_loss: 0.0555 - val_accuracy: 0.9850
Epoch 2/10
44/44 [==============================] - 0s 5ms/step - loss: 0.0363 - accuracy: 0.9904 - val_loss: 0.0460 - val_accuracy: 0.9883
Epoch 3/10
44/44 [==============================] - 0s 5ms/step - loss: 0.0246 - accuracy: 0.9940 - val_loss: 0.0444 - val_accuracy: 0.9883
Epoch 4/10
44/44 [==============================] - 0s 5ms/step - loss: 0.0110 - accuracy: 0.9982 - val_loss: 0.0451 - val_accuracy: 0.9817
Epoch 5/10
44/44 [==============================] - 0s 5ms/step - loss: 0.0107 - accuracy: 0.9972 - val_loss: 0.0465 - val_accuracy: 0.9883
Epoch 6/10
44/44 [==============================] - 0s 5ms/step - loss: 0.0071 - accuracy: 0.9976 - val_loss: 0.0497 - val_accuracy: 0.9883
Epoch 7/10
44/44 [==============================] - 0s 5ms/step - loss: 0.0078 - accuracy: 0.9946 - val_loss: 0.0506 - val_accuracy: 0.9833
Epoch 8/10
44/44 [=

We found that the performance of this classifier was about 82% at the hyperparameter values specified.

This is higher than the baseline of bag-of-words combined with logistic regression, and approximately equal to sent2vec combined with logistic regression.

## Dual Task Experiment

We now introduce another task the task of determining if an email is spam or not. 

Assuming the availability of the `sent2vec` vectors `train_x2` corresponding to the emails in the data sample, It shows how one can create a multipleoutput
shallow neural model to train it simultaneously for email spam classification and the classification of IMDB movie reviews via hard-parameter sharing.

In [49]:
input1_shape = (len(train_x[0]))
input2_shape = (len(train_x2[0]))

sent2vec_vectors1 = Input(shape=input1_shape)
sent2vec_vectors2 = Input(shape=input2_shape)

combined = concatenate([sent2vec_vectors1, sent2vec_vectors2])

dense1 = Dense(512, activation="relu")(combined)
drop_out = Dropout(0.3)(dense1)

output1 = Dense(1, activation="sigmoid", name="classification1")(drop_out)
output2 = Dense(1, activation="sigmoid", name="classification2")(drop_out)

model = Model(inputs=[sent2vec_vectors1, sent2vec_vectors2], outputs=[output1, output2])

In [55]:
# Specify two loss functions (both binary_crossentropy in our case)
model.compile(loss={"classification1": "binary_crossentropy", "classification2": "binary_crossentropy"}, 
              optimizer="adam", metrics=["accuracy"])
# Specify training and validation data for each input
history = model.fit([train_x, train_x2], [train_y, train_y2], validation_data=([test_x, test_x2], [test_y, test_y2]),
                    batch_size=32, epochs=10, shuffle=True)

Epoch 1/10
44/44 [==============================] - 1s 14ms/step - loss: 1.0890 - classification1_loss: 0.6867 - classification2_loss: 0.4024 - classification1_accuracy: 0.5805 - classification2_accuracy: 0.8378 - val_loss: 0.6158 - val_classification1_loss: 0.5356 - val_classification2_loss: 0.0802 - val_classification1_accuracy: 0.7400 - val_classification2_accuracy: 0.9817
Epoch 2/10
44/44 [==============================] - 0s 7ms/step - loss: 0.5355 - classification1_loss: 0.4742 - classification2_loss: 0.0613 - classification1_accuracy: 0.8008 - classification2_accuracy: 0.9888 - val_loss: 0.5419 - val_classification1_loss: 0.4856 - val_classification2_loss: 0.0562 - val_classification1_accuracy: 0.7533 - val_classification2_accuracy: 0.9817
Epoch 3/10
44/44 [==============================] - 0s 8ms/step - loss: 0.4453 - classification1_loss: 0.4083 - classification2_loss: 0.0370 - classification1_accuracy: 0.8326 - classification2_accuracy: 0.9952 - val_loss: 0.5817 - val_classif

## Conclusions

We found, upon training the multi-task system, that the IMDB classification performance dropped slightly from approximately 82% in the single-task shallow setup to about 80%. The email classification accuracy similarly dropped from 98.7% to 98.2%.

First of all, observe that trained model can be deployed independently for each task, by simply replacing the omitted task input with zeros in order to respect the expected overall input dimension, and ignoring the corresponding output. Moreover, we expect the shared pretrained layer dense1 of the multi-task setup to be more readily generalizable to arbitrary new tasks. This is because it has been trained to be predictive on a more varied and general set of data and tasks.

To make this more concrete, consider replacing either or both task-specific layers with new dense1 ones, initializing the shared layer to pretrained weights from the experiment above and fine-tuning the resulting model on the new task dataset. Having seen a broader range of task data, potentially similar to the newly added tasks, these shared weights are more likely to
contain useful information for the downstream tasks being considered.